In [1]:
from IPython.display import display, HTML      # wide screen
display(HTML(data=""" <style>     div#notebook-container { width: 95%;} div#menubar-container { width: 85%; } div#maintoolbar-container { width: 99%; } </style> """))

import os
import sys
import platform
import math
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy
import flopy.utils.binaryfile as bf
from flopy.export.shapefile_utils import shp2recarray
from rasterio import Affine
from flopy.utils.reference import SpatialReference
import shapefile    # not used directly here but for some reason flopy needs this to do shapefile stuff and BTW its not import pyshp as you would think its import shapefile, dumb...
import pandas as pd
import fiona
from rasterio.features import rasterize
import rasterio
import rasterio.mask
import geopandas
from osgeo import gdal

flopy is installed in C:\Users\cshuler\AppData\Local\Continuum\anaconda3\envs\ASPA_Framework\lib\site-packages\flopy


#### Numpy array convert to .asc file


In [2]:
# take a numpy array from this model and turn it into a .asc raster based on the model geometry dimensions
def nparray_to_asc(input_array, output_raster, ncol, nrow, xll, yll, cellsize):

    np.savetxt(output_raster, input_array)

    new_first = ('NCOLS {}\n'                        # these are the parameters for the .asc file
                 'NROWS {}\n'
                 'XLLCENTER {}\n'
                 'YLLCENTER {}\n'
                 'CELLSIZE {}\n'
                 'NODATA_value -999.0\n'.format(ncol,nrow, xll, yll, np.mean([delc, delr]) ))  

    with open(output_raster, 'r+') as file:                # add in new first line and save file  
        file_data = file.read()
        file. seek(0, 0)
        file. write(new_first + '\n' + file_data)

In [5]:
# model geometry from regional model
# params to pass to Model class
delr = 169 
delc = 165
nlay = 1
nrow = 100
ncol = 200
botm = -1250
xll = 515244    # hard code too
yll = 8410178

# Manuall moved over the .hds file, this utility will extract the heads
hds = bf.HeadFile('modelface_PYPCGA.hds')
times = hds.get_times() # simulation time, steady state
head = hds.get_data(totim=times[-1])

In [6]:
cellsize = np.mean([delc, delr])

headsqeeze = np.squeeze(head)

nparray_to_asc(headsqeeze, "PyPCGA_model_Heads.asc", ncol=ncol, nrow=nrow, xll=xll, yll=yll, cellsize=cellsize)